# [실습] HuggingFace에서 공개 모델 사용하기

https://huggingface.co 의 링크에서 공개된 모델을 사용하는 방법을 알아볼 것  
큰 모델은 양자화를 통해 불러오기 가능  
실행 전 T4 GPU를 선택하고 실행(RTX 3060과 비교되는 colab 가장 작은 GPU)

In [ ]:
!pip install accelerate huggingface_hub transformers bitsandbytes==0.45.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.6 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.44.1
    Uninstalling bitsandbytes-0.44.1:
      Successfully uninstalled bitsandbytes-0.44.1


In [ ]:
!pip install langchain langchain-community langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


허깅페이스 모델에 접속하기 위해, 허깅페이스 라이브러리 불러오기  
Llama, Gemma와 같이 계정별 인증이 필요한 경우는 토큰 인증이 필수 (https://huggingface.co/login?next=%2Fsettings%2Ftokens)  
Read 토큰 생성하고 활용  
본 실습은 Qwen 모델을 사용하므로 꼭 인증이 필요하지는 않음 (https://huggingface.co/Qwen/Qwen2.5-7B-Instruct)

In [ ]:
from huggingface_hub import login

login(token='...')

모델과 토크나이저를 불러오기  
토크나이저는 모델의 구동에 필수적이며, 모델별 Special Token이나 Chat Template 등의 요소가 포함되어 있음  
따라서 모델을 저장할 때는 토크나이저도 항상 같이 저장할 것

In [ ]:
!nvidia-smi

Sat May 31 04:40:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

T4 GPU 15GB vRAM 확인 가능  
불러오려는 Qwen2.5-7B-Instruct 모델은 7B니까 곱하기를 2를 한 14GB의 vRAM을 가중치만 불러오는데 사용하게 되므로 여유 부족함  
환경보다 큰 모델을 사용하려면 양자화 처리

In [ ]:
from transformers import BitsAndBytesConfig

import transformers
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델의 주소를 입력
model_id = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype='auto',  # 가중치의 데이터 타입 자동 감지
    quantization_config=quantization_config,
    # 양자화란? 큰 모델의 가중치를 작게 만드는 방법
    # 32비트 모델은 *4 16비트 모델은  *2를 하여 기본 필요 파라미터 확인
    # 이미 양자화된 모델을 불러올 때는 config.json에 양자화 옵션이 들어가므로 quantization_config를 안 넣어도 됨
    device_map={"": 0},  # 0번 GPU에 할당
  )

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

모델 구조 확인

In [ ]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), 

In [ ]:
model.save_pretrained("Qwen2.5-7B-Instruct")

In [ ]:
tokenizer.save_pretrained("Qwen2.5-7B-Instruct")

('Qwen2.5-7B-Instruct/tokenizer_config.json',
 'Qwen2.5-7B-Instruct/special_tokens_map.json',
 'Qwen2.5-7B-Instruct/chat_template.jinja',
 'Qwen2.5-7B-Instruct/vocab.json',
 'Qwen2.5-7B-Instruct/merges.txt',
 'Qwen2.5-7B-Instruct/added_tokens.json',
 'Qwen2.5-7B-Instruct/tokenizer.json')

모델의 generation을 잘 수행하기 위해 text-generation pipeline을 사용  
기존 모델에 들어가던 매개변수(max_new_tokens 등)은 여기 포함됨

In [ ]:
# 아래 옵션은 주로 모델 페이지의 generate_config에서 참고할 수 있음
# ref: https://huggingface.co/Qwen/Qwen2.5-7B-Instruct/blob/main/generation_config.json

# 출력 방향 설정
gen_config = {
  "bos_token_id": 151643,
  "pad_token_id": 151643,
  "do_sample": True,  # 확률 기반 샘플링 (False면 최대 확률 토큰만 선택)
  "eos_token_id": [
    151645,
    151643
  ],
  "repetition_penalty": 1.05,  # 토큰 반복 억제 페널티
  "temperature": 0.7,  # temperature를 높게 주면 중국어 토큰이 많이 나옴
  "top_p": 0.8,  # 누적 확률 기준 80% 내 토큰만 샘플링
  "top_k": 20,  # Top 20 토큰만 샘플링
  "transformers_version": "4.37.0"
}

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,  # 프롬프트를 포함하여 출력할 것인지 결정(default: True)
    **gen_config
)

Device set to use cuda:0


랭체인과 연동을 위해 langchain_huggingface 모듈 사용

In [ ]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

llm = HuggingFacePipeline(pipeline=pipe, pipeline_kwargs=gen_config)

chat_model = ChatHuggingFace(llm=llm, tokenizer=tokenizer)

In [ ]:
# Tokenizer에 저장된 Template을 기반으로 질문 답변 생성
chat_model.invoke("트럼프가 누구야?")

AIMessage(content=' Donald Trump는 2017년부터 2021년까지 미국의 45대 대통령을 역임한 정치인입니다. 그는 비즈니스맨, 토목 건설업자, 텔레비전 프로듀서 등 다양한 분야에서 활동하며 이름을 알렸습니다. 2016년 대선에서 공화당 후보로 선출되며 대통령으로 당선되었습니다. 그의 경제 정책, 보호무역주의, 이민 정책 등에 대한 입장을 중심으로 많은 논란이 있었습니다.', additional_kwargs={}, response_metadata={}, id='run--999a4934-dc27-4c1d-b003-df49d83b02cc-0')

In [ ]:
chat_model.invoke("Who is Trump?")

AIMessage(content='Donald John Trump is a well-known American businessman, real estate developer, and former politician who served as the 45th President of the United States from January 20, 2017, to January 20, 2021. He was born on June 14, 1946, in Queens, New York.\n\nTrump has been a prominent figure in American politics and business for decades. Prior to his presidency, he ran a successful real estate and development company and became known for his reality television show "The Apprentice," where he served as the executive producer and host. His business ventures have included hotels, casinos, golf courses, and other real estate projects across the United States and internationally.\n\nAs president, Trump\'s policies and actions were often controversial and polarizing. He was known for his unconventional approach to governance, frequent use of social media, and his stance on various domestic and international issues. After leaving office, Trump remains a significant political figu

불러온 모델의 종류에 따라, 토크나이저에서 Special Tokens와 Chat Template을 확인 가능

In [ ]:
tokenizer.all_special_tokens

['<|im_end|>',
 '<|endoftext|>',
 '<|im_start|>',
 '<|object_ref_start|>',
 '<|object_ref_end|>',
 '<|box_start|>',
 '<|box_end|>',
 '<|quad_start|>',
 '<|quad_end|>',
 '<|vision_start|>',
 '<|vision_end|>',
 '<|vision_pad|>',
 '<|image_pad|>',
 '<|video_pad|>']

In [ ]:
print(tokenizer.chat_template)  # 동적 변수 치환을 위한 템플릿 언어 jinja2

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are Qwen, created by Alibaba C

포맷 메세지는 토크나이저를 통해 변환 가능

In [ ]:
system_msg = "Please answer in Korean."
question = "마틴 스콜세이지의 대표작 3개는 무엇인가요? Please answer in Korean."

msg = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": question},
]

prompt = tokenizer.apply_chat_template(msg, tokenizer=False, add_generation_prompt=True)
print(prompt)

[151644, 8948, 198, 5501, 4226, 304, 16134, 13, 151645, 198, 151644, 872, 198, 125544, 144305, 141767, 144063, 41429, 12802, 21329, 20401, 60960, 126414, 67511, 220, 18, 59761, 16560, 132779, 31328, 19969, 35711, 30, 5209, 4226, 304, 16134, 13, 151645, 198, 151644, 77091, 198]


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

chat_template = ChatPromptTemplate(
    [
        ('system', '{instruction}'),
        ('user', '{question}')
    ]
)

chat = chat_template | chat_model | StrOutputParser()

In [ ]:
print(chat.invoke({'instruction': system_msg, 'question': question}))

마틴 스콜세지의 대표작 3개는 다음과 같습니다:

1. 깊은 바다의 노래 (Goodfellas, 1990)
2. 토르토바시오 (😎, 1990)
3. 카페 디비전 (Casino, 1995)


채팅 모델에서는 토크나이저를 연결해야 하나 ChatHuggingFace에서는 그대로 실행해도 됨

In [ ]:
summarize_instruction = '''아래 글을 읽고 200자 이내 요약문을 작성하세요.'''

example_news = '''[경제] 'AI 기술 도입으로 기업 생산성 30% 향상, 혁신의 새로운 장을 열다.'
최근 국내 주요 기업들이 인공지능(AI) 기술을 도입한 이후, 생산성이 평균적으로 30% 이상 증가했다.
그러나 모든 기업이 이러한 성공 사례를 재현한 것은 아니며, 일부 기업들은 AI 기술 적응 과정에서 실패했다.
정부 역시 AI 기술의 확산과 도입을 촉진하기 위해 다양한 지원책을 마련하고 있으며, 특히 중소기업 혜택을 제공한다.'''

chat.invoke({'instruction': summarize_instruction, 'question': example_news})

'국내 주요 기업들의 AI 도입으로 생산성이 30% 증가했으나, 모든 기업이 성공하지 못했다. 정부는 AI 도입을 지원하며 중소기업에도 혜택을 주고 있다.'

7B은 SLM 중 큰 편, LLM 중 작은 편